In [1]:
print("hello")

hello


In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# template = """Question: {question}

# Answer: Let's think step by step."""

# prompt = ChatPromptTemplate.from_template(template)

# model = OllamaLLM(model="llama3")

# chain = prompt | model

# response = chain.invoke({"question": "How do I pick outfit for today?"})

In [5]:
# print(response)

Let's break it down together.

Step 1: Consider the occasion. What are you planning to do today? Are you going to work, school, running errands, or something else?

Please respond with your answer, and we'll move on to the next step!


In [9]:
import sqlite3
import random
import string
from datetime import datetime, timedelta

# Create SQLite database and connection
conn = sqlite3.connect('properties.db')
cursor = conn.cursor()

# Create table with the selected schema
cursor.execute('''CREATE TABLE IF NOT EXISTS properties (
    property_id TEXT,
    list_price REAL,
    status TEXT,
    city TEXT,
    list_date TEXT
)''')

# Function to generate random dummy data
def random_string(length=10):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())),
    )

# Insert 20 dummy records
for _ in range(20):
    property_id = random_string(10)
    list_price = round(random.uniform(100000, 1000000), 2)
    status = random.choice(["active", "sold", "pending"])
    city = random.choice(["Los Angeles", "New York", "Houston", "Miami"])
    list_date = random_date(datetime(2021, 1, 1), datetime(2024, 8, 1)).strftime("%Y-%m-%d")

    # Insert the dummy record into the database
    cursor.execute('''INSERT INTO properties (property_id, list_price, status, city, list_date) 
                      VALUES (?, ?, ?, ?, ?)''',
                   (property_id, list_price, status, city, list_date))

# Commit and close the connection
conn.commit()
conn.close()


In [10]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('properties.db')
cursor = conn.cursor()

# Query to select all records from the properties table
cursor.execute('''SELECT * FROM properties''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Print the column names
columns = [description[0] for description in cursor.description]
print(" | ".join(columns))

# Print each row in the table
for row in rows:
    print(" | ".join(str(value) for value in row))

# Close the connection
conn.close()


property_id | list_price | status | city | list_date
EE3KUSWMD9 | 709533.83 | sold | Miami | 2021-06-09
3QW39WRTTL | 901498.28 | sold | Miami | 2024-02-09
YOA0336IMP | 390973.57 | active | New York | 2022-07-30
Y1LLDGS0C3 | 348274.1 | active | Miami | 2023-07-13
NQVXHOS74P | 207258.9 | sold | New York | 2022-02-11
69N9769441 | 395974.56 | pending | Houston | 2023-10-17
YYEFU7QPWH | 307788.58 | active | New York | 2022-01-09
V89LEPHSPR | 360683.07 | sold | Houston | 2021-07-15
P2GU7D5EK4 | 924844.74 | active | Los Angeles | 2024-05-20
KWCH2P2X0I | 581983.98 | active | New York | 2021-08-28
HCM4CHT8MQ | 128762.77 | sold | Miami | 2022-08-08
5P4FJD4ST0 | 475173.47 | active | Miami | 2024-06-08
WDAQWL7M40 | 831178.72 | sold | Los Angeles | 2021-12-30
P8YF99O7ZE | 784826.76 | pending | Miami | 2022-07-21
0AS8KV90R8 | 210300.21 | pending | Miami | 2023-10-18
DN7LFWJNGG | 516470.42 | active | Los Angeles | 2021-10-25
GVJXEH7AU0 | 327436.07 | pending | Los Angeles | 2021-03-10
52XD7PUXVP | 896

In [21]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///properties.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM properties LIMIT 10;")

sqlite
['properties']


"[('EE3KUSWMD9', 709533.83, 'sold', 'Miami', '2021-06-09'), ('3QW39WRTTL', 901498.28, 'sold', 'Miami', '2024-02-09'), ('YOA0336IMP', 390973.57, 'active', 'New York', '2022-07-30'), ('Y1LLDGS0C3', 348274.1, 'active', 'Miami', '2023-07-13'), ('NQVXHOS74P', 207258.9, 'sold', 'New York', '2022-02-11'), ('69N9769441', 395974.56, 'pending', 'Houston', '2023-10-17'), ('YYEFU7QPWH', 307788.58, 'active', 'New York', '2022-01-09'), ('V89LEPHSPR', 360683.07, 'sold', 'Houston', '2021-07-15'), ('P2GU7D5EK4', 924844.74, 'active', 'Los Angeles', '2024-05-20'), ('KWCH2P2X0I', 581983.98, 'active', 'New York', '2021-08-28')]"

In [22]:
llm = OllamaLLM(model="llama3")

In [31]:
from langchain.chains import create_sql_query_chain


chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "Which is the most expensive property?"})


In [32]:
x = response.split(":")[-1].strip()

In [33]:
db.run(x)

"[('P2GU7D5EK4', 924844.74)]"